# Optimizing instruments with Cython

This tutorial demonstrates how to convert a "pure Python" instrument -- an instrument defined within a python environment such as a .py script or this running notebook -- into a C-compiled instrument that can run significantly faster.  It should be noted that in many cases pure Python instruments run fast enough to not need optimization. However, if instruments are complex, or require 'expensive' unit generators, the time it takes to render the audio file can become an impediment to the project at hand. In cases like this it makes sense to clone the instrument source code to a new file, compile that file down to a C shared library using Python's [Cython compiler](https://cython.org/), and import the optimized instrument into the Python.

This notebook assumes that Cython is installed in your virtual environment. If you aren't sure, use this next cell to determine if you need to use pip to install it before proceeding.

In [ ]:
! which cython cythonize

Notebook imports:

In [ ]:
import sys
sys.path.append("/Users/taube/Software/musx")
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from IPython.display import Code

import numpy as np
import pysndlib.clm as clm
import pysndlib.sndlib as snd
import pysndlib.instruments as ins
import musx

print(f"musx version: {musx.version}")

## A pure-python instrument

The candidate for optimization is an [additive synthesis](https://ccrma.stanford.edu/~jos/pasp/Additive_Synthesis.html) instrument called `ccbell()` that produces the sound of ten different bells that were installed in the Cathedral Church of Saint Michael (Coventry Cathedral) in Coventry, England in 1774. Tragically, the bells were destroyed along with the cathedral during a German blitzkrieg on November 14, 1940. However, acoustic measurements in the 1920s produced enough information for it to be possible to reconstruct the sounds using modern audio synthesis techniques. See: [https://www.hibberts.co.uk/coventry-cathedral-old-ten/](https://www.hibberts.co.uk/coventry-cathedral-old-ten/) for more information.

<!-- The candidate instrument for optimization is called `ccbell()` and it uses [additive synthesis](https://ccrma.stanford.edu/~jos/pasp/Additive_Synthesis.html) to genreate the sound of ten unique bells that were installed in Cathedral Church of Saint Michael (Coventry Cathedral) in Coventry, England in 1774. The bells were destroyed along with the cathedral during a German blitzkrieg on November 14, 1940. See: [https://www.hibberts.co.uk/coventry-cathedral-old-ten/](https://www.hibberts.co.uk/coventry-cathedral-old-ten/) for more information. 
-->

An additive synthesis instrument generates complex waveforms by summing the frequencies, phases and amplitudes of its constituent partials. This simple summation can produce very high quality results, but often at considerable cost due to the fact that each partial requires its own oscillator and amplitude envelope. As more partials are added the closer the instrument's sound comes to the actual source sound but at the expense of additional computation and longer wait times.

<!--
The consequence of this is that as more partials are added the closer the instrument's sound comes to the actual source sound but only at the cost of additional computation and longer wait times.


This notebook uses an [Additive Synthesis](https://ccrma.stanford.edu/~jos/pasp/Additive_Synthesis.html) instrument as its optimization target. This technique can model any complex waveform by summing the frequencies, phases and amplitudes of the target's consituent partials. The result produces very high quality sounds, but at considerable cost due to the fact that each partial requires an oscilltor and amplitude envelope. As more partials are added the closer the instrument sound comes to the actual souce sound but the more compution the processess requires.

uses [additive synthsis] to resurrect the sound of the original 10 bells of the Cathedral Church of Saint Michael (Coventry Cathedral) in Coventry, England. The bells were destroyed along with the cathedral during a German blitzkrieg on November 14, 1940. See: [https://www.hibberts.co.uk/coventry-cathedral-old-ten/](https://www.hibberts.co.uk/coventry-cathedral-old-ten/)
-->

Evaluate this cell to define the pure Python `ccbell()` instrument. Be sure to look over the instrument's code and comments before moving on:

In [ ]:
# coventry.py
# A resurrection of Coventry Cathedral's 10 bells that were destroyed along with
# the cathedral during a German blitzkrieg on November 14, 1940.
# See: https://www.hibberts.co.uk/coventry-cathedral-old-ten/

import numpy as np
import pysndlib.clm as clm
import pysndlib.sndlib as snd
import pysndlib.instruments as ins

# A dictionary storing the spectral information of seven bell partials (hum,
# prime, tierce, quint, nominal, superquint, and octave-nominal) for each of the
# ten Coventry bells. The partials are indexed high-to-low by integer id's
# 1 to 10, and their frequencies in the table have all been 'normalized' to the
# Prime harmonic (2nd partial) so the bells can be tuned (shifted) to frequencies
# other than their original tunings.  

coventry_bells = {
# Bell Id  Hum      Prime Tierce  Quint   Nominal Superquint Octave-Nominal
    1:     [0.6076, 1.0,  1.3304, 1.8727, 2.2176, 3.2756,    4.4376],
    2:     [0.5988, 1.0,  1.3007, 1.844,  2.156,  3.1733,    4.3033],
    3:     [0.5932, 1.0,  1.3327, 1.7515, 2.2325, 3.3006,    4.491],
    4:     [0.5911, 1.0,  1.2961, 1.7712, 2.1615, 3.2019,    4.3872],
    5:     [0.6042, 1.0,  1.3056, 1.7604, 2.1481, 3.162,     4.3009],
    6:     [0.572,  1.0,  1.2537, 1.639,  2.0537, 3.022,     4.139],
    7:     [0.5583, 1.0,  1.2333, 1.6611, 2.0556, 3.0639,    4.2139],
    8:     [0.5096, 1.0,  1.1699, 1.5137, 1.9055, 2.8096,    3.8479],
    9:     [0.5757, 1.0,  1.2368, 1.6924, 2.0263, 2.9868,    4.0888],
    10:    [0.5608, 1.0,  1.2099, 1.5996, 1.9683, 2.903,     3.9718]
}


# A dictionary storing the prime frequencies of the original (destroyed) bells.

coventry_primes = {1: 620.5, 2: 557, 3: 499, 4: 483, 5: 432, 
                   6: 410, 7: 360, 8: 365, 9: 304, 10: 283.5}


# Amplitude envelopes for each bell partial, adapted from data on Hibberts'
# original website (no longer available).

coventry_amplitudes = [
  # SplashAmp,   TailAmp,  Attack,  Decay   
    [0.0,        0.1408,   0.3,     1.0   ], # Hum
    [0.0,        0.704,    0.3,     0.4615], # Prime
    [0.0,        0.7916,   0.0,     0.1714], # Tierce
    [0.0833,     0.0317,   0.25,    0.08  ], # Quint
    [1.0,        0.25,     0.1,     0.3429], # Nominal
    [0.292,      0.1083,   0.5,     0.12  ], # Superquint
    [0.15,       0.0617,   0.11,    0.1333]  # Octave Nominal
]


# The ccbell() instrument sounds a coventry bell given the bell's id number 
# 1 to 10 and the frequency in hertz of that bell's prime harmonic. If freq 
# is 0 the sound of the original (destroyed) bell will be heard.

def ccbell(beg, bell=1, dur=6, freq=0, amp=0.3, deg=45, dist=0, rev=0):
    if bell < 1 or bell > 10:
        raise ValueError(f"Bell id {bell} is not between 1 and 10 inclusive.")
    if freq <= 0:
        freq = coventry_primes[bell]
    # Frequencies for all seven partials given the frequency of the prime harmonic.
    partials = [p * freq for p in coventry_bells[bell]]
    start = clm.seconds2samples(beg)
    end   = start + clm.seconds2samples(dur)
    location = clm.make_locsig(degree=deg, distance=dist, reverb=rev)
    radians, envarray = [], []
    # gather phase and amp envelopes for each partial
    for i, p in enumerate(partials):
        hertz = round(p, 3)
        amps = coventry_amplitudes[i]
        splashamp = round(amps[0] * amp, 3)
        tailamp = round(amps[1] * amp, 3)
        attack = round(amps[2] if amps[2] else .001, 3)
        decay = round(amps[3] * dur, 3)
        envl =  [0, splashamp, attack/4, tailamp/1.5, attack, tailamp, attack+decay, 0]
        radians.append(clm.hz2radians(hertz))
        envarray.append(clm.make_env(envl, scaler=amp, duration=dur))
    # store everything in a pysnlib 'oscil_bank'
    bank_size = len(radians)
    phase_array = np.zeros(bank_size)
    amps_array = np.zeros(bank_size)
    envs_array = np.array(envarray)
    # use an oscil bank to hold the oscil/envelope pairs for each partial
    oscils = clm.make_oscil_bank(radians, phase_array, amps_array)
    for i in range(start, end):
        for j in range(bank_size):
            amps_array[j] = clm.env(envs_array[j])
        clm.locsig(location, i, clm.oscil_bank(oscils))

print(f"ccbell: {ccbell}")

<!-- from support.coventry import ccbell

Code('./support/coventry.py', language='python') -->

#### Example 1

This example performs the ten bells sounding their original pitches. Bells are identified using integers with 1 representing the highest bell:

In [ ]:
bells = [i for i in range(1, 11)]
print(f"bell ids: {bells}")

with clm.Sound("test.wav", play=True, statistics=True):
    for t, i in enumerate(bells):
        ccbell(t*2, bell=i)

#### Example 2

This longer example performs a 60 note sequence with the bells set to 5-TET tuning (5 equal steps per octave, each step ~2.4 semitones wide).  The bell sequence starts with descending bells, followed by four peals in which all but the lowest bell are randomly ordered, and ending with a retrograde of the opening. A slight variance is added to start times, durations, and amplitudes so bell partials 'line up' in a realistic way:

<!-- This example performs the 10 bells in 5-TET tuning (5 equal steps per octave, each step 2.4 semitones).  The 60-note peal (sequence) is a concatenation of the descending bells, followed by several rounds in which all but the lowest bell are randomly reordered, and ending with a retrograde of the opening. A slight variance is added to start times, durations, and amplitudes so bell partials 'line up' in a more realistic way. -->

Be sure to notice the amount of time it takes for cbell() to render the audio file on your computer. As a reference point, on a 2021 Apple M1 Pro laptop it takes about 9 seconds... :

In [ ]:
def bongs(ids, reps):
    # randomly order the top 9 bells keeping the lowest bell last.
    return sum( [musx.shuffle(ids, end=len(ids)-1) for _ in range(reps)], [])

def coda(ids):
    # ascend through the top 9 bells keeping the lowest bell last.
    return ids[-2::-1] + ids[-1:]

def jiggle(value, amt=.025):
    # returns small fluctation + or - a given value.
    return musx.vary(value, amt)

bells = [i for i in range(1, 11)]
tunings = {i: round( musx.hertz("b5") * (2 ** (- (i-1)/5)) ) for i in bells}
peal = bells + bongs(bells, 4) + coda(bells)

print(f"bell ids: {bells}")
print(f"tunings: {tunings}")
print(f"strikes: {len(peal)}")
print(f"peal: {peal}")

with clm.Sound("test.wav", play=True, statistics=True): #reverb=ins.nrev(decay_time=8), 
    b = 0
    for bell in peal:
        freq = tunings[bell]
        #amp, dur = (.6, 8) if bell == 10 else (.35, 4)
        amp, dur = (.6, 7) if bell == 10 else (.35, 3.5)
        ccbell(b, bell=bell, freq=freq, dur=jiggle(dur), amp=jiggle(amp)) #, rev=.01, dist=5
        b += jiggle(.5)

<!--

Optimizing the all-python cbell instrument involves several steps. 

1.  Copy the instrument source code to a new source file for optimization.
2.  Compile instrument code into a C shared using Cython that can be imported into a Pyton sessions
3.  Add variable typing to the instrument source code to gain more speed

coventry.py -> coventry.pyx -> coventry.c => coventry.so
-->

#### Example 3

This last example of the "pure Python" instrument performs a famous bell ringing pattern called Plain Bob, which swaps pairs of bells on each peal to produces 21 permutations of the bells without repeating. On ten bells, the complete pattern takes 210 strikes. For more infomation see [change-ringing](https://en.wikipedia.org/wiki/Change_ringing),  [Plain Bob](https://www.thomasalspaugh.org/pub/crg/plainBob.html), the [Rotation pattern](https://github.com/musx-admin/musx/blob/main/docs/patterns.html) and the musx [coventry demo](https://github.com/musx-admin/musx/blob/main/demos/coventry.ipynb).


In [ ]:
def jiggle(value, amt=.1):
    # returns small fluctation + or - a given value.
    return musx.vary(value, amt)
    
with clm.Sound("test.wav", play=True, statistics=True, reverb=ins.nrev(decay_time=2.5)):
    t, d, a  = 0, .23, .6
    plain_bob = musx.Rotation([b for b in range(1,11)], [[0, 2, 1], [1, 2, 1]]).all(wrapped=True)
    print(f'strikes: {len(plain_bob)}')
    print(f'pattern: {plain_bob}')
    for i, b in enumerate(plain_bob):
        ccbell(t, bell=b, dur=jiggle(1), amp=jiggle(a), rev=0.01)
        t += jiggle(d)

<!-- ##ORIGINAL
#bellnames = [i for i in range(1,11)]
#plain_bob = musx.Rotation(bellnames, [[0, 2, 1], [1, 2, 1]]).all(wrapped=True) 
#numbells = len(plain_bob)
#ldeg = [0, 45, numbells*.25, 45, numbells*.75, 0 ]
#rdeg = [0, 45, numbells*.25, 45, numbells*.75, 90]
#print(f"{numbells} bells in pattern, cranking...")
#top, bot = bellnames[0],bellnames[-1]
#hilite = [top, bot]

#with clm.Sound(play=True, channels=2, reverb=ins.nrev(decay_time=6), statistics=True):
#    beg = 0
#    for i,b in enumerate(plain_bob):
#        dur = musx.between(1.4, 3.8)
#        amp = musx.between(.1,.15)
#        ccbell(beg=beg, bell=b,
#                 dur=dur if b not in hilite else dur * {top:1.5, bot: 2.25}[b],
#                 freq=0,
#                 amp=amp*1.1 if b not in hilite else amp * {top:1.8, bot:2.3}[b], #top:1.75, bot:2.35
#                 deg=musx.between(musx.interp(i, ldeg), musx.interp(i, rdeg)),
#                 dist=musx.interp(i, 0, 2,  numbells/2,  2,  numbells, 15),   
#                 rev=musx.interp(i, 0, .001,numbells/2, .03, numbells, .06) )
#        beg += musx.between(.27,.30)
->

## Optimizing the ccbell() instrument

### Step 1: copy coventry.py to coventry.pyx

This next cell takes the 'pure python' ccbell() source code defined above and saves it in a file called "coventry.pyx" located in the support/ subfolder of this notebook's directory, e.g. "musx/tutorials/support/coventry.pyx". Python uses the .pyx file extension to distinguish between 'regular' python file (.py) and python files to be compiled by Cython:

In [ ]:
for s in _ih:
    if s.startswith('# coventry.py'):
        with open("./support/coventry.pyx", "w") as pyxfile:
            pyxfile.write(s)
print(f"wrote {pyxfile.name}")

<!-- ### Step 2: create setup.py

Evaluate the next cell to create a './support/setup.py' script that Python will use to build projects and packages.  In our case, the script will pass the 'coventry.pyx' file to Cython for compilation to a C sharded library that can be imported into a python session. --?

### Step 2: Compile coventry.pyx with Cython

There are a number of ways to interface with Cython; this notebook uses the most direct way, by calling the `cythonize` command line tool.  Cythonize will take the .pyx file, convert it to a C text file (coventry.c) and compile the C file into shared library. In the command line below, the `--3str` argument informs Cython that the file is Python3 and `--inline` places the newly compiled instrurment (a C shared library) in the same directory as the .pyx file. Note that the compilation process will generate some warnings, suppressing these is possible using a Python 'setup.py' build script, but is beyond the scope of this tutorial:

In [ ]:
! cythonize --3str --inplace support/coventry.pyx

### Step 3: Load the compiled C instrument into Python

Check to make sure the shared library was created and is located in the support directory along with 'coventry.pyx' and 'coventry.c'.  The compete name of this shared library depends on the operating system and python build but will start with 'coventry':

In [ ]:
! ls support/coventry*

Use Python's standard import statement to load the compiled ccbell() instrument. Notice that after importing the variable `ccbell`  now points to a 'cyfunction' and not a 'function':

In [ ]:
from support.coventry import ccbell

print(f"ccbell: {ccbell}")

Test the compiled version of ccbell:

In [ ]:
with clm.Sound("test.wav", play=True, statistics=True): #reverb=ins.nrev(decay_time=8), 
    b = 0
    for bell in peal:
        freq = tunings[bell]
        amp, dur = (.6, 8) if bell == 10 else (.35, 4)
        ccbell(b, bell=bell, freq=freq, dur=jiggle(dur), amp=jiggle(amp)) #, rev=.01, dist=5
        b += jiggle(.5)

## Optimizing using Cython datatypes

Significant speedup can be achieved for some instruments using Cython's C datatypes to statically type critical variables and functions, for example, those involved with the instument's run time (sample-by-sample) `for` loop.

In [ ]:
PYX CODE

For more information about compiling and optimizing Pure Python code, see Cython's [Basic Tutorial](https://cython.readthedocs.io/en/latest/src/tutorial/cython_tutorial.html) and Peter Baumgartens [An Introduction to Just Enough Cython to be Useful](https://www.peterbaumgartner.com/blog/intro-to-just-enough-cython-to-be-useful/).

Optional cleanup:

In [ ]:
! rm -r ./support/coventry.c ./support/coventry.cpython* ./support/build